## Bajar la base

In [ ]:
#Si no está descargado, es importante bajar el paquete pymrio
#!pip install pymrio   
import pandas as pd
import numpy as np
import os 

In [ ]:
os.getcwd()

In [ ]:
os.chdir("...your path...")
os.listdir()

## Matriz de importaciones

In [ ]:
M=pd.read_excel('mip_matriz4.xls')
M_1=M.iloc[6:202,2:]
M_1

In [ ]:
#indice en filas
index=M_1.loc[7:202]
index=index.iloc[:,0]
M_2=M_1.loc[7:202].set_index(index)
del M_2['Unnamed: 2']
M_2

### Demanda intermedia de m

In [ ]:
#Nombres en columnas
M_3=M.iloc[6:7,3:127].values[0] #sectores demanda interna
M_3=list(M_3)
M_3=pd.Index(M_3)
M_3

In [ ]:
#generar valores por columnas de demanda de importaciones
M_di=M.iloc[7:202,3:127].sum()
M_di.reindex(M_3)
M_di.index=M_3
M_di

In [ ]:
#chequeo
M.iloc[6:202,3].head(5)
print(M.iloc[6:7,3],M.iloc[7:202,3].sum())

### Demanda final de m

In [ ]:
M.iloc[4:,124:].head(10)

In [ ]:
X=M.iloc[7:202,130] # son 153 productos de los 195 porque el resto son servicios!

C=M.iloc[7:202,133]

G=M.iloc[7:202,135]

I=M.iloc[7:202,137]


for x in [C,G,I,X]:
    print(x.sum(), len(x))
    
#demanda final
Fm=pd.concat([C,G,I,X],axis=1)
dicty={key: value for key,
            value in zip(Fm.columns, ['C','G','I', 'X'])}
Fm.rename(dicty, axis=1, inplace=True)
Fm

## Coeficientes directos e indirectos (Inversa de Leontief)

In [ ]:
L=pd.read_excel('mip_matriz14.xls')
L_3=list(L.iloc[6:7,2:127].values[0])#sectores demanda interna
L_3

In [ ]:
L_1=L.iloc[7:131,2:127]

#filas
L_1.index=L_3    
L_1

#columnas
L_1.columns=L_3
L_1

## Economia doméstica

In [ ]:
Z=pd.read_excel('mip_matriz12.xls')
Z.iloc[6:131,1:126]

#demanda final doméstica
X=Z.iloc[7:131,129] 

C=Z.iloc[7:131,132]

G=Z.iloc[7:131,134]

I=Z.iloc[7:131,136]


for x in [C,G,I,X]:
    print(x.sum(), len(x))
    
Fd=pd.concat([C,G,I,X],axis=1)
dicty={key: value for key,
            value in zip(Fd.columns, ['C','G','I', 'X'])}
Fd.rename(dicty, axis=1, inplace=True)
Fd.sum()

#demanda intermedia doméstica

Z_3=list(L.iloc[6:7,2:126].values[0])
Z_1=Z.iloc[7:131,2:126]
Z_1.index=Z_3
Z_1.columns=Z_3
Z_1=Z_1.sum()

In [ ]:
Fd.index=L_3
Fd

### Coeficientes de importación

In [ ]:
m_identidad=np.identity(len(L_3))  #matrix identidad
m_identidad


vector_cm=pd.DataFrame([Z_1, M_di], index=('Z_1','M_di')).transpose()
vector_cm['cm']=vector_cm['M_di']/(vector_cm['M_di']+vector_cm['Z_1'])
vector_cm.loc[vector_cm['cm'].isna()]=0
del vector_cm['Z_1']
del vector_cm['M_di']
m=np.array(vector_cm['cm'])*m_identidad
m

M_di.shape

## Importaciones indirectas

[Bussiere et al (2013)](https://www.aeaweb.org/articles?id=10.1257/mac.5.3.118)

$M_{ind}=m(I-A)^{-1}F_d$

In [ ]:
Mind=np.matmul(np.matmul(m,L_1),Fd)
Mind

## Importaciones directas

[Bussiere et al (2013)](https://www.aeaweb.org/articles?id=10.1257/mac.5.3.118)

Son las filas de venta de sectores del ROW a los componentes de la demanda final doméstica

$M_{dir}=F_m$

In [ ]:
#Tengo 146 productos, deberia clasificarlos segun los sectores para que sean 124.Ver en el excel matriz_4b_trabajando
#primer intento con los 195 productos de las impo porque en C hay un servicio de transporte de pasajeros
#que representa el 14% de las impo del secTor. No contemplarlo sesga la estimacion a nivel aggregado
Fm

## Intensidad importadora de los componentes de demanda

[Bussiere et al (2013)](https://www.aeaweb.org/articles?id=10.1257/mac.5.3.118)

$w_k=\frac{uMind_k+uMdir_k}{uFm_k+uFd_k}$  

donde `u` un vector de unos

In [ ]:
h_1124=np.array([1 for v in range(len(L_3))], dtype=int) #creo un vector horizontal de 1
h_1195=np.array([1 for v in range(M_2.shape[0])], dtype=int) #creo un vector vertical de 1

print(len(h_1124), len(h_1195))
h_1195.shape

In [ ]:
demanda=['C','G','I']
w=[]
for x in demanda:
 w.append((np.matmul(h_1124,Mind[x])+np.matmul(h_1195,np.array(Fm[x])))/(np.matmul(h_1124,Fd[x])+np.matmul(h_1195,np.array(Fm[x]))))



In [ ]:
#exportaciones las pongo aparte porque no aparecen en Mdir
w.append((np.matmul(h_1124,Mind['X']))/(np.matmul(h_1124,Fd['X'])))   

In [ ]:
w

In [ ]:
wdir=[]
for x in demanda:
 wdir.append((np.matmul(h_1195,Fm[x]))/(np.matmul(h_1124,Fd[x])+np.matmul(h_1195,Fm[x])))


In [ ]:
wdir

In [ ]:
wind=[]
for x in demanda:
 wind.append((np.matmul(h_1124,Mind[x]))/(np.matmul(h_1124,Fd[x])+np.matmul(h_1195,Fm[x])))

wind.append((np.matmul(h_1124,Mind['X']))/(np.matmul(h_1124,Fd['X'])))    #para las expo

In [ ]:
wind

In [ ]:
demanda=['C','G','I','X']
df1 = pd.DataFrame(wind,index=demanda)
df1.rename({0: 'wind'}, axis=1, inplace=True)

In [ ]:
demanda=['C','G','I']
df2 = pd.DataFrame(wdir,index=demanda)
df2.rename({0: 'wdir'}, axis=1, inplace=True)

In [ ]:
demanda=pd.Index(['C','G','I','X'])
df3 = pd.DataFrame(w,index=demanda)
df3.rename({0: 'wgt'}, axis=1, inplace=True)

In [ ]:
df=df3.join(df1.join(df2))
df.wdir[df['wdir'].isna()==True]=0
df

In [ ]:
import os
os.chdir('...your path...')

In [ ]:
df.to_excel("wgt_mip97.xlsx")